# Dask DataFrame (1)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Макрушин С.В. Лекция "Dask DataFrame"
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.memory_usage_per_partition.html#dask.dataframe.DataFrame.memory_usage_per_partition
* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.map_partitions.html#dask.dataframe.DataFrame.map_partitions
* https://docs.dask.org/en/stable/generated/dask.array.histogram.html
* https://docs.dask.org/en/stable/dataframe-categoricals.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.pivot_table.html#dask.dataframe.DataFrame.pivot_table

* https://docs.dask.org/en/stable/best-practices.html
* https://docs.dask.org/en/stable/dashboard.html
* https://distributed.dask.org/en/stable/client.html

## Задачи для совместного разбора

1\. Создать `dask.DataFrame`. Рассмотреть основные возможности (выбор строк, работа с датами, добавление столбцов)

In [1]:
import dask

2\. Используя метод pipe, добавьте столбец с полом человека и нормализуйте столбцы x и y

3\. При помощи сводных таблиц посчитайте для каждой пары (gender, key) среднее значение по столбцу x.

## Лабораторная работа 12

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений проводятся над `dask.DataFrame` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.DataFrame` к `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.DataFrame` (то есть то, что было получено до вызова `compute`, а не после).

<p class="task" id="1"></p>

1\. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты. Выведите на экран информацию о количестве сегментов и типе столбцов. Выведите на экран 5 первых и 5 последних строк таблицы. В случае сообщения об ошибке объясните причину и исправьте ошибку.

In [31]:
import dask.dataframe as dd
import pandas as pd

In [32]:
recipes = dd.read_csv('recipes/recipes_full_*.csv', parse_dates=['submitted'], dtype={'minutes': 'float64',
       'n_steps': 'float64'})

In [33]:
recipes.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 8 entries, id to n_ingredients
dtypes: datetime64[ns](1), object(2), float64(2), int64(3)

In [34]:
recipes.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10


In [35]:
recipes.tail()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
278949,1029131,tuti waffle snackies steakhouse,19.0,171345,1973-10-18,4.0,"according to a providence journal article, ama...",4
278950,1700703,noelias cheats crocante fleisch zitumbuwa,1.0,30228,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1
278951,1910650,rubbed restuffed pelmeni bedouin flavourful,60.0,591905,2009-09-26,3.0,another great recipe to add to the growing swe...,2
278952,713836,stems polpettine peezi,NaN,357389,2003-09-30,4.0,adapted from top secret recipes. love this!,9
278953,660699,clementines,64.0,29196,1973-06-03,6.0,this would make a great start to your holiday ...,8


<p class="task" id="2"></p>

2\. Выведите на экран следующую информацию:
* количество сегментов в таблице
* объем используемой памяти каждого сегмента
* количество строк в каждом сегменте
* количество строк во всей таблице.

In [38]:
print(f'количество сегментов в таблице:{recipes.npartitions}')
print('объем используемой памяти каждого сегмента:')
recipes.memory_usage_per_partition().compute()

количество сегментов в таблице:8
объем используемой памяти каждого сегмента:


0    17853248
1    17853248
2    17853248
3    17853248
4    17853248
5    17853184
6    17853184
7    17853184
dtype: int64

In [40]:
print(f'количество строк в каждом сегменте:{recipes.map_partitions(len).compute()}')

количество строк в каждом сегменте:0    278955
1    278955
2    278955
3    278955
4    278955
5    278954
6    278954
7    278954
dtype: int64


In [41]:
print(f'количество строк во всей таблице:{recipes.shape[0].compute()}')

количество строк во всей таблице:2231637


<p class="task" id="3"></p>

3\. Воспользовавшись индексатором `loc`, отберите строки фрейма `recipes`, имеющие индекс 42. Выведите полученные строки и их количество на экран. Объясните результат в виде текстового комментария.

In [45]:
idk = recipes.loc[42]
print(idk.compute())
print("их количество:", len(idk))


         id                                               name  minutes   
42   223847                                      gyros burgers     25.0  \
42  1218643                        zazu thisclose portage dahi     39.0   
42   265832                                     alber ensalada     35.0   
42  1972307  eclectic rigamarta bechamel bananarita chipiro...     30.0   
42  1116892  reid pot meatza roseland trouble gougere tunes...     18.0   
42   249625                  african peanut and ginger chicken    120.0   
42   452814                                    pepper jack mac     45.0   
42  1601792   fetasaganaki kalaloo kabos unbaked jhal decision     57.0   

    contributor_id  submitted  n_steps   
42          451301 2007-04-21      5.0  \
42           68357 1993-04-13      5.0   
42         1155210 1976-02-17      3.0   
42           82616 1996-11-29      1.0   
42         1389351 2003-07-24      3.0   
42          317934 2007-08-29     17.0   
42          174096 2011-04-03 

мы получили с каждого сегмента строку с 42 индексом. Всего строк 8 тк сегментов 8 и в каждом сегменте только один инедкс 42

<p class="task" id="4"></p>

4\. Выясните, сколько рецептов содержат слово `chocolate` в названии (`name`). Выведите на количество уникальных идентификаторов авторов таких рецептов.

In [49]:
import re

In [55]:
count_chocolate = recipes['name'].str.contains('chocolate', case=False).sum().compute()
count_chocolate

11628

In [57]:
unique_contributors = recipes.loc[recipes['name'].str.contains('chocolate', case=False).fillna(False), 'contributor_id'].nunique().compute()


In [58]:
unique_contributors

4457

<p class="task" id="5"></p>

5\. При помощи функции `da.histogram` посчитайте значения для построения гистограммы для столбца `n_ingredients`. Визуализируйте полученные результаты при помощи функции `plt.bar`. Добавьте на рисунок 2 вертикальные линии, соответствующие квантилям уровней 0.25 и 0.75. Сделайте масштаб вертикальной оси логарифмическим.

Допускается вычисление статистики по столбцу (применение метода `compute`) до того, как будет вызван `da.histogram`. 

In [52]:
import dask.array as da
import matplotlib.pyplot as plt

<p class="task" id="6"></p>

6\. Напишите функции:
* `minutes_to_hours(df)`, которая принимает на вход фрейм и заменяет столбец `minutes` на столбец `hours` с соответствующим преобразованием чисел;
* `extract_ymd(df)`, которая добавляет 3 столбца: `year`, `month` и `day`, полученные на основе столбца `submitted`;
* `add_month_name(df)`, которая добавляет категориальный столбец `month_name` с названием месяца из даты и вызывает для него метод `.cat.as_known()`. 

Создайте фрейм `recipes_pipe` путем цепочки вызовов метода `pipe` с применением данных функций. Выведите на экран строку, соответствующую рецепту с id=683970. Для отбора строки воспользуйтесь методом `query`.

<p class="task" id="7"></p>

7\. Используя результат предыдущей задачи, постройте сводную таблицу при помощи метода `pivot_table`, где по строкам располагаются года, по столбцам - названия месяцев, а в ячейках содержится средняя длина рецептов (в часах) в данный год и месяц. Выведите таблицу на экран. Выведите на экран информацию за апрель 1970-1975 годов (включительно).

<p class="task" id="8"></p>

8\. В архиве `recipes_additional.zip` находятся файлы, содержащие информацию об рецептах блюд, полученные из другого источника. В связи с этим названия и количество столбцов в этих данных отличаются от того, что находится в архиве `recipes_full.zip`.

Объедините два набора набора данных (добавьте строки из одного набора данных к другому), согласовав названия столбцов и форматы данных. Итоговый фрейм должен содержать следующие колонки: `id`, `name`, `minutes`, `contributor_id`, `submitted`, `n_steps`, `description`, `n_ingredients`, `#tags` и `view_30_days`. Добавьте столбец `dset` с информацией о том, из какого датасета была получены данные.

Выведите на экран количество пропусков в каждом из столбцов полученной таблицы и общее количество строк в ней.